In [1]:
import os
import numpy as np

from pathlib import Path

from kbmod.fake_data_creator import *
from kbmod.run_search import *
from kbmod.search import *

# Setup file paths and create data

There are at least two file paths you need to setup in order to run kbmod:
1. The im_filepath provides a path to the input images.
1. The res_filepath provides a path to the directory where the output results will be stored.

A time and psf file can optimally be specified.

If you already have data files, you can use those. Or for the purposes of the notebook, you can generate fake data as in the create_fake_data.ipynb notebook, which is what we do in the code block below. If you want to use your own data simply override these two path variables.


In [2]:
im_filepath = "./fake_data"
ds = FakeDataSet(256, 256, 20)
ds.insert_random_object(500)
ds.save_fake_data(im_filepath)

print(os.listdir(im_filepath))

['000008.fits', '000009.fits', '000010.fits', '000011.fits', '000012.fits', '000013.fits', '000014.fits', '000015.fits', '000016.fits', '000017.fits', '000018.fits', '000019.fits', '000000.fits', '000001.fits', '000002.fits', '000003.fits', '000004.fits', '000005.fits', '000006.fits', '000007.fits']


In [3]:
res_filepath = "./fake_results"
if not Path(res_filepath).is_dir():
    print(f"Directory {res_filepath} does not exist. Creating.")
    os.mkdir(res_filepath)

# Run KBMOD

In [4]:
results_suffix = "DEMO"
v_min = 0  # Pixels/day
v_max = 50
# Offset by PI for prograde orbits in lori allen data
ang_below = -np.pi + np.pi / 2.0  # Angle below ecliptic
ang_above = np.pi + np.pi / 2.0  # Angle above ecliptic
v_steps = 50
ang_steps = 50

v_arr = [v_min, v_max, v_steps]
ang_arr = [ang_below, ang_above, ang_steps]

num_obs = 15

input_parameters = {
    # Required
    "im_filepath": im_filepath,
    "res_filepath": res_filepath,
    "time_file": None,
    "output_suffix": results_suffix,
    "v_arr": v_arr,
    "ang_arr": ang_arr,
    # Important
    "num_obs": num_obs,
    "do_mask": True,
    "lh_level": 10.0,
    "gpu_filter": True,
    # Fine tuning
    "sigmaG_lims": [15, 60],
    "mom_lims": [37.5, 37.5, 1.5, 1.0, 1.0],
    "peak_offset": [3.0, 3.0],
    "chunk_size": 1000000,
    "stamp_type": "cpp_median",
    "eps": 0.03,
    "clip_negative": True,
    "mask_num_images": 10,
}

rs = run_search(input_parameters)
rs.run_search()

---------------------------------------
Loading Images
---------------------------------------
Loaded 20 images
Times set
Starting Search
---------------------------------------
Ecliptic Angle = 1.1901
Min. Search Angle = 2.7609
Max Search Angle = 5.9025
Min Velocity = 0.0000
Max Velocity = 50.0000
Using in-line GPU sigmaG filtering methods
sigmaG limits: [15,60]
sigmaG coeff: 0.7753
Search finished in 0.451s
---------------------------------------
Retrieving Results
---------------------------------------
Getting results...
---------------------------------------
Chunk Start = 0
Chunk Max Likelihood = 4.80
Chunk Min. Likelihood = -1.00
---------------------------------------
Extracted batch of 0 results for total of 0
---------------------------------------
Applying Stamp Filtering
---------------------------------------
Skipping. Nothing to filter.
---------------------------------------
Saving Results
---------------------------------------
Time taken for patch:  1.1431975364685059


# Clean up the fake data

If fake data was generated, you probably want to clean it up here. Do **not** delete the directory if you are using static data.

In [5]:
# Uncomment to delete the fake data.
# ds.delete_fake_data(im_filepath)